In [1]:
import pandas as pd
import plotly.express as px 
from urllib.request import urlopen
import json
import janitor
import plotly.io as pio
import pandas_geojson as pdg
pio.renderers.default='notebook'

In [2]:
asthmaca = pd.read_csv('asthma_redownload_utf8.csv')
asthmaca

,COUNTY,YEAR,STRATA,STRATA NAME,AGE GROUP,NUMBER OF HOSPITALIZATIONS,AGE-ADJUSTED HOSPITALIZATION RATE,COMMENT
0,California,2015,Total population,All ages,All ages,"27,262",7.0,NaN
1,Alameda,2015,Total population,All ages,All ages,"1,435",9.3,NaN
2,Alpine,2015,Total population,All ages,All ages,0,0.0,NaN
3,Amador,2015,Total population,All ages,All ages,28,7.5,NaN
4,Butte,2015,Total population,All ages,All ages,143,6.7,NaN
...,...,...,...,...,...,...,...,...
4479,Tulare,2020,Race/ethnicity,Multi-race,All ages,0,0.0,NaN
4480,Tuolumne,2020,Race/ethnicity,Multi-race,All ages,0,0.0,NaN
4481,Ventura,2020,Race/ethnicity,Multi-race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
4482,Yolo,2020,Race/ethnicity,Multi-race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...


In [3]:
asthmaca.dtypes

COUNTY                                object
YEAR                                   int64
STRATA                                object
STRATA NAME                           object
AGE GROUP                             object
NUMBER OF HOSPITALIZATIONS            object
AGE-ADJUSTED HOSPITALIZATION RATE    float64
COMMENT                               object
dtype: object

In [4]:
asthmaca = asthmaca.clean_names()
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment
0,California,2015,Total population,All ages,All ages,"27,262",7.0,NaN
1,Alameda,2015,Total population,All ages,All ages,"1,435",9.3,NaN
2,Alpine,2015,Total population,All ages,All ages,0,0.0,NaN
3,Amador,2015,Total population,All ages,All ages,28,7.5,NaN
4,Butte,2015,Total population,All ages,All ages,143,6.7,NaN
...,...,...,...,...,...,...,...,...
4479,Tulare,2020,Race/ethnicity,Multi-race,All ages,0,0.0,NaN
4480,Tuolumne,2020,Race/ethnicity,Multi-race,All ages,0,0.0,NaN
4481,Ventura,2020,Race/ethnicity,Multi-race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...
4482,Yolo,2020,Race/ethnicity,Multi-race,All ages,NaN,NaN,Count and rate suppressed in accordance with d...


In [5]:
asthmaca = asthmaca.query('strata == \'Total population\'')
asthmaca = asthmaca.query('strata_name == \'All ages\'')
asthmaca = asthmaca.query('county != \'California\'')
asthmaca = asthmaca.query('year != 2015')
asthmaca = asthmaca.query('year != 2020')
asthmaca['county'] = asthmaca['county'] + ' County' #for mapping FIPS
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment
709,Alameda County,2016,Total population,All ages,All ages,904,5.9,NaN
710,Alpine County,2016,Total population,All ages,All ages,0,0.0,NaN
711,Amador County,2016,Total population,All ages,All ages,13,3.4,NaN
712,Butte County,2016,Total population,All ages,All ages,78,4.0,NaN
713,Calaveras County,2016,Total population,All ages,All ages,10,NaN,Rate not available due to statistical instability
...,...,...,...,...,...,...,...,...
2886,Tulare County,2019,Total population,All ages,All ages,210,4.2,NaN
2887,Tuolumne County,2019,Total population,All ages,All ages,21,5.4,NaN
2888,Ventura County,2019,Total population,All ages,All ages,252,3.0,NaN
2889,Yolo County,2019,Total population,All ages,All ages,57,2.8,NaN


In [6]:
geocodes = pd.read_excel('all-geocodes-v2017.xlsx', header=4)
geocodes = geocodes.clean_names()
geocodes

,summary_level,state_code_fips_,county_code_fips_,county_subdivision_code_fips_,place_code_fips_,consolidtated_city_code_fips_,area_name_including_legal_statistical_area_description_
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County
...,...,...,...,...,...,...,...
43905,50,72,145,0,0,0,Vega Baja Municipio
43906,50,72,147,0,0,0,Vieques Municipio
43907,50,72,149,0,0,0,Villalba Municipio
43908,50,72,151,0,0,0,Yabucoa Municipio


In [7]:
fipmap = geocodes.set_index('area_name_including_legal_statistical_area_description_')['county_code_fips_'].to_dict()
asthmaca['fip'] = asthmaca['county'].map(fipmap)
asthmaca

,county,year,strata,strata_name,age_group,number_of_hospitalizations,age_adjusted_hospitalization_rate,comment,fip
709,Alameda County,2016,Total population,All ages,All ages,904,5.9,NaN,1
710,Alpine County,2016,Total population,All ages,All ages,0,0.0,NaN,3
711,Amador County,2016,Total population,All ages,All ages,13,3.4,NaN,5
712,Butte County,2016,Total population,All ages,All ages,78,4.0,NaN,19
713,Calaveras County,2016,Total population,All ages,All ages,10,NaN,Rate not available due to statistical instability,9
...,...,...,...,...,...,...,...,...,...
2886,Tulare County,2019,Total population,All ages,All ages,210,4.2,NaN,107
2887,Tuolumne County,2019,Total population,All ages,All ages,21,5.4,NaN,109
2888,Ventura County,2019,Total population,All ages,All ages,252,3.0,NaN,111
2889,Yolo County,2019,Total population,All ages,All ages,57,2.8,NaN,113


In [9]:
geojson = pdg.read_geojson('California_County_Boundaries.geojson')
geojson.get_properties()

['OBJECTID',
 'CountyName',
 'AdminRegion',
 'FireMAR',
 'LawMAR',
 'State_FIPS_ID',
 'County_FIPS_ID',
 'Shape__Area',
 'Shape__Length']